In [1]:
import numpy as np
import pandas as pd
import textwrap


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# TensorFlow and transformers
import tensorflow as tf
from transformers import BertTokenizer, TFAutoModelForSequenceClassification



c:\Users\Simon Andersen\miniconda3\envs\BigData\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def print_row(data, idx, target_column, symbols_per_line=100):
    """
    Print a row of a DataFrame with word wrapping

    Parameters
    ----------
    data : DataFrame
        The DataFrame containing the data to be printed
    idx : int
        The index of the row to be printed
    target_column : str
        The name of the column to be printed
    symbols_per_line : int
        The number of symbols per line at which to wrap the text
    
    Returns
    -------
    None
        
    """
    value = str(data[target_column].iloc[idx])

    # Remove excess spaces
    value = ' '.join(value.split())
    
    # Wrap text while respecting word boundaries
    wrapped_value = textwrap.fill(value, width=symbols_per_line)
    
    print(wrapped_value)
    return

def print_list(list, elements_per_line=10):
    """
    Print a list with a specified number of elements per line
    """

    list_range = range(0, len(list), elements_per_line)

    for i in list_range:
        print(*list[i:i+elements_per_line])
    return

# Define a function to find unique words in a text
def find_unique_words(text, word_list):
    """
    Find unique words in a text. Append the unique words to a list.
    """
    for word in text.split():
        if word not in word_list:
            word_list.append(word)
    return

# print table summary with total samples in each dataset, positive samples, and negative samples
def pretty_print_info_table(labels_train, labels_test, validation=False):

    val_str = 'Val.' if validation else 'Test'
    data_for_table = {
        'train': {
            'total': len(labels_train),
            'pos': np.sum(labels_train),
            'neg': len(labels_train) - np.sum(labels_train)
        },
        'test': {
            'total': len(labels_test),
            'pos': np.sum(labels_test),
            'neg': len(labels_test) - np.sum(labels_test)
        }
    }
    data_for_table['total'] = {
        'total': data_for_table['train']['total'] + data_for_table['test']['total'],
        'pos': data_for_table['train']['pos'] + data_for_table['test']['pos'],
        'neg': data_for_table['train']['neg'] + data_for_table['test']['neg']
    }

    print("""
    Info table:
    +----------------+---------+---------+-------+
    |                | Training| {}    | Total
    +----------------+---------+---------+-------+
    | Total samples  | {:7d} | {:7d} | {:7d} |
    | Pos. samples   | {:7d} | {:7d} | {:7d} |  (AI generated)
    | Neg. samples   | {:7d} | {:7d} | {:7d} |  (Human written)
    """.format(
        val_str,
        data_for_table['train']['total'],
        data_for_table['test']['total'],
        data_for_table['total']['total'],
        data_for_table['train']['pos'],
        data_for_table['test']['pos'],
        data_for_table['total']['pos'],
        data_for_table['train']['neg'],
        data_for_table['test']['neg'],
        data_for_table['total']['neg'],
        
        ))
    return

1 = AI-generated

In [4]:
data = pd.read_csv('ai_human_training_data.csv', index_col=0)
data['label'] = data['label'].astype(int)
data['text'] = data['text'].astype(str)

data_test = pd.read_csv('ai_human_test_data.csv', index_col=0)
data_test['label'] = data_test['label'].astype(int)

pretty_print_info_table(data.label, data_test.label, validation=False)


    Info table:
    +----------------+---------+---------+-------+
    |                | Training| Test    | Total
    +----------------+---------+---------+-------+
    | Total samples  |    2854 |     215 |    3069 |
    | Pos. samples   |    1321 |     103 |    1424 |  (AI generated)
    | Neg. samples   |    1533 |     112 |    1645 |  (Human written)
    


In [ ]:
for i in range(0, 5):
    print("AI-generated text? ", bool(data['label'].iloc[i]))
    print("Text:")
    print_row(data, i, 'text', symbols_per_line=100)
    print("\n")

In [5]:
test_texts = data_test['text'].astype(str)
test_labels = data_test['label'].astype(int)

# Split data into training and test sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42
)

pretty_print_info_table(train_labels, val_labels, validation=True)


    Info table:
    +----------------+---------+---------+-------+
    |                | Training| Val.    | Total
    +----------------+---------+---------+-------+
    | Total samples  |    2283 |     571 |    2854 |
    | Pos. samples   |    1062 |     259 |    1321 |  (AI generated)
    | Neg. samples   |    1221 |     312 |    1533 |  (Human written)
    


In [6]:
model_name = 'bert-base-multilingual-cased'
model_name = 'bert-base-uncased'
max_length_of_input = 256

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name, use_fast=True)

# Tokenize the text inputs for BERT
def tokenize_texts(texts, max_len=max_length_of_input):
    return tokenizer(
        texts.tolist(), 
        padding=True, 
        truncation=True, 
        max_length=max_len, 
        return_tensors='tf'
    )

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)
test_encodings = tokenize_texts(test_texts)

In [7]:
# Load pre-trained BERT model for binary classification
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Convert labels to TensorFlow format
train_labels_tf = tf.convert_to_tensor(train_labels.values)
val_labels_tf = tf.convert_to_tensor(val_labels.values)
test_labels_tf = tf.convert_to_tensor(test_labels.values)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_encodings['input_ids'],
    train_labels_tf,
    validation_data=(val_encodings['input_ids'], val_labels_tf),
    epochs=1,  
    batch_size=512
)


In [16]:
# Predict on the test set
logits = model.predict(test_encodings['input_ids']).logits
predictions = np.argmax(logits, axis=1)

# Evaluate the model
accuracy = accuracy_score(test_labels, predictions)
report = classification_report(test_labels, predictions, target_names=['AI', 'Human'])

print(f"Test Accuracy: {accuracy}")
print("Classification Report:\n", report)


1/1 [==============================] - 4s 4s/step


ValueError: Number of classes, 1, does not match size of target_names, 2. Try specifying the labels parameter